# Recurrent Neural Networks: Programming Practice

COSC 410: Applied Machine Learning\
Colgate University\
*Prof. Apthorpe*

## Overview

This notebook will give you practice with the following topics:
  1. Creating and training simple RNNs
  2. Creating and training LSTM & GRU networks

We will be using a new **natural language** dataset of IMDB movie reviews. We will attempt to perform a classification task to label reviews as having either *positive* or *negative* sentiment. The reviews have already been preprocessed from words --> numbers, such that the most common word is encoded as 1, the second most common word is encoded as 2, etc. 

## Part 1. Data Import & Inspection

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks

In [2]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

(x_train, y_train), (x_val, y_val) = ks.datasets.imdb.load_data(num_words=max_features) # Load the dataset

x_train = ks.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) # Pad reviews with 0s to equal length
x_val = ks.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen) # Pad reviews with 0s to equal length

print(len(x_train), "Training sequences") # Print number of training reviews
print(len(x_val), "Validation sequences") # Print number of validation reviews

print(np.unique(y_train, return_counts=True)[1]) # print counts of classes in training set
print(np.unique(y_val, return_counts=True)[1]) # print counts of classes in validation set

17473536/17464789 [==============================] - 1s 0us/step
25000 Training sequences
25000 Validation sequences
[12500 12500]
[12500 12500]


**Note:** One of the benefits of RNNs is that they can handle sequences of differing lengths, so why do we pad the training and validation data to equal lengths? ... Training time! Keras is optimized for mini-batch gradient descent when the training examples are non-ragged matrices (a.k.a. *tensors*...hence "Tensorflow"). Once we have trained the model, we can perform predictions on *new* reviews of arbitary length.  

In [7]:
# Function that converts an embedded review back into English bag-of-words representation
def reverse_embedding(review):
    words_to_ints = ks.datasets.imdb.get_word_index(path="imdb_word_index.json")
    reverse_embedding_map = dict(zip(words_to_ints.values(), words_to_ints.keys()))
    return " ".join([reverse_embedding_map.get(x, "") for x in review])

# Function that prints the label (1 = positive review, 0 = negative review) and bag-of-words text of a review
def print_example_and_label(idx):
    print(f"Label: {y_train[idx]}")
    print(reverse_embedding(x_train[idx]) + "\n")
    
print_example_and_label(0)
print_example_and_label(1)

Label: 1
1654784/1641221 [==============================] - 0s 0us/step
to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have two of script their with her n

## Part 2. 1-Node RNNs

We'll start by creating three different 1-Node RNNs with `SimpleRNN`, `LSTM`, and `GRU` nodes. 

In [9]:
# SimpleRNN model
model = ks.Sequential([
    ks.Input(shape=(None,)),
    ks.layers.Embedding(max_features, 1),
    ks.layers.SimpleRNN(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
782/782 [==============================] - 21s 26ms/step - loss: 0.6938 - accuracy: 0.5366 - val_loss: 0.6735 - val_accuracy: 0.5992
Epoch 2/5
782/782 [==============================] - 17s 22ms/step - loss: 0.6531 - accuracy: 0.6426 - val_loss: 0.6509 - val_accuracy: 0.6142
Epoch 3/5
782/782 [==============================] - 17s 22ms/step - loss: 0.6199 - accuracy: 0.6817 - val_loss: 0.6369 - val_accuracy: 0.6249
Epoch 4/5
782/782 [==============================] - 17s 22ms/step - loss: 0.5916 - accuracy: 0.7094 - val_loss: 0.6303 - val_accuracy: 0.6368
Epoch 5/5
782/782 [==============================] - 17s 22ms/step - loss: 0.5673 - accuracy: 0.7299 - val_loss: 0.6284 - val_accuracy: 0.6424


In [10]:
# LSTM model
model = ks.Sequential([
    ks.Input(shape=(None,)),
    ks.layers.Embedding(max_features, 1),
    ks.layers.LSTM(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
782/782 [==============================] - 35s 43ms/step - loss: 0.6869 - accuracy: 0.5718 - val_loss: 0.6654 - val_accuracy: 0.6524
Epoch 2/5
782/782 [==============================] - 31s 40ms/step - loss: 0.6352 - accuracy: 0.6806 - val_loss: 0.6208 - val_accuracy: 0.6670
Epoch 3/5
782/782 [==============================] - 26s 34ms/step - loss: 0.5855 - accuracy: 0.7215 - val_loss: 0.5925 - val_accuracy: 0.6855
Epoch 4/5
782/782 [==============================] - 27s 34ms/step - loss: 0.5460 - accuracy: 0.7518 - val_loss: 0.5724 - val_accuracy: 0.7073
Epoch 5/5
638/782 [=======================>......] - ETA: 4s - loss: 0.5128 - accuracy: 0.7759

KeyboardInterrupt: 

In [11]:
# GRU model
model = ks.Sequential([
    ks.Input(shape=(None,)),
    ks.layers.Embedding(max_features, 1),
    ks.layers.GRU(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
782/782 [==============================] - 39s 47ms/step - loss: 0.6847 - accuracy: 0.5674 - val_loss: 0.6720 - val_accuracy: 0.6149
Epoch 2/5
782/782 [==============================] - 36s 46ms/step - loss: 0.6494 - accuracy: 0.6570 - val_loss: 0.6423 - val_accuracy: 0.6276
Epoch 3/5
782/782 [==============================] - 47s 60ms/step - loss: 0.6094 - accuracy: 0.6967 - val_loss: 0.6199 - val_accuracy: 0.6409
Epoch 4/5
782/782 [==============================] - 52s 67ms/step - loss: 0.5744 - accuracy: 0.7185 - val_loss: 0.6063 - val_accuracy: 0.6516
Epoch 5/5
782/782 [==============================] - 53s 67ms/step - loss: 0.5451 - accuracy: 0.7371 - val_loss: 0.5985 - val_accuracy: 0.6660


## Part 3. Recurrent Layer Hyperparameters

`SimpleRNN`, `LSTM` and `GRU` layers share several hyperparameters that can be set via keyword arguments. The documentation for each of these layers is here: https://keras.io/api/layers/recurrent_layers/
  * `return_sequences`: Whether the layers should return a single value for each input sequence (`False`, *default*) or a new multi-step sequence for each input sequence (`True`)
  * `kernel_regularizer`, `bias_regularizer`, and `recurrent_regularizer` for L1, L2, or L1L2 regularization
  * `dropout` and `recurrent_dropout` for setting dropout rates of forward and recurrent connections. 

## Part 4: Stacked RNNs

Finally, let's try a stacked LSTM network with 2 hidden layers. 

In [ ]:
# Stacked LSTM model
model = ks.Sequential([
    ks.Input(shape=(None,)),
    ks.layers.Embedding(max_features, 1),
    ks.layers.LSTM(64, return_sequences=True)
    ks.layers.LSTM(64, return_sequences=True)
    ks.layers.LSTM(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

We can see that the stacked RNN has higher plateau performance and reaches that performance more quickly. However, the increasing difference between train accuracy and val accuracy indicates that we are in an overfitting regime, so it doesn't make sense to increase the size of the network further. 